In [1]:
from lp_random_gen import random_lp_int
from docplex.mp.advmodel import AdvModel
from docplex.mp.model_reader import ModelReader
from helpers import *
from qiskit.optimization.algorithms.optimization_algorithm import OptimizationResultStatus
DIR = 'SYNTH_DATA/BINARY_SPARSE/'
getPath = lambda fn : DIR+fn+".lp"

# Create Test Data

In [2]:
NUM_TESTS = 10
num_constr_per_connection = 3
num_vars_per_connection = 5
lp_bin = random_lp_int(num_constr_per_connection, num_vars_per_connection, 0, 1)

# create problems of size 5 , 10 ,15 , ... , 50
for i in range(1,NUM_TESTS +1):
    
    model = lp_bin.createLP("test_bin_"+str(i), i)
    with open(getPath(lp_bin.name), 'w+') as file:
        file.write(model.export_as_lp_string())

# Solve with DWave Systems

In [3]:
from dwave.system import LeapHybridSampler
# we dont use DWaveCliqueSampler because the problem is sparse

In [4]:
num_reads = 2000
qps = createModelsFromDir(DIR)

In [5]:
with open(getPath("results"), 'w+') as file:
    file.write("Start Tests \n \n")  

In [6]:
found_no_solution_ct = 0
for qp_name in sorted(qps.keys(), key = lambda name : int(name[name.rfind('_')+1:])) :
    results = {}
    solvers = {"Leap_Hybrid" : createSolver(LeapHybridSampler()) , "DWaveSampler" : createSolver(num_reads = num_reads)}
    
    for sname in solvers.keys():
        optimizer = solvers[sname]
        try:
            results[sname] = optimizer.solve(qps[qp_name])
        except Exception as ex:
            print(qp_name, " ", sname, type(ex).__name__, " : ", ex)
    if any(res.status == OptimizationResultStatus.SUCCESS for res in results.values()):
        found_no_solution_ct = 0
        with open(getPath("results"), 'a+') as file:
            file.write("\n ------------------------------------- \n"+qp_name)
            for sname in solvers.keys():
                file.write("\n"+sname)
                if sname not in results:
                    file.write("\n Exception occurred")
                else:
                    res = results[sname]
                    if res.status == OptimizationResultStatus.SUCCESS:
                        cres  = qps[qp_name].model.solve()
                        if res.fval != cres.get_objective_value():
                            file.write("\n optimal value DWave   "+str(res.fval) + "  ,  cplex:"+ str(cres.get_objective_value()))
                            file.write("\n"+str(res.x)+"   cplex:"+str(cres.get_all_values()))
                        else:
                            file.write("\n found solution!")
                    else:
                        file.write("\n"+str(res.status))   
    else:
        # test if cres can find a solution
            
        cres  = qps[qp_name].model.solve()
        with open(getPath("results"), 'a+') as file:
            file.write("\n Cres : "+str(cres))
        if cres.is_valid_solution():
            found_no_solution_ct = found_no_solution_ct + 1
        if found_no_solution_ct == 2:
            break

constraint A0_leb0 is infeasible due to substitution
constraint A0_leb2 is infeasible due to substitution
constraint A1_leb0 is infeasible due to substitution
constraint A1_leb2 is infeasible due to substitution
constraint A2_leb0 is infeasible due to substitution
constraint A2_leb1 is infeasible due to substitution
constraint A2_leb2 is infeasible due to substitution
constraint A0_leb0 is infeasible due to substitution
constraint A0_leb1 is infeasible due to substitution
constraint A1_leb1 is infeasible due to substitution
constraint A2_leb0 is infeasible due to substitution
constraint A2_leb1 is infeasible due to substitution
constraint A3_leb0 is infeasible due to substitution
constraint A3_leb1 is infeasible due to substitution
constraint A3_leb2 is infeasible due to substitution
constraint A0_leb0 is infeasible due to substitution
constraint A0_leb1 is infeasible due to substitution
constraint A1_leb1 is infeasible due to substitution
constraint A1_leb2 is infeasible due to substi

Result:
Leap finds solutions but they are not the best
DWave Sampler finds solutions up to 3 x 5 vars